Start!

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import time

In [ ]:
import astropy.units as u
import numpy as np
import pandas as pd
import astropy.constants as c

In [ ]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj
from dolan_junction import DolanJunction as junction
import Default_params as dp
from pocket import TransmonPocket as pocket


In [ ]:
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [ ]:
gui = MetalGUI(dp.design)
design = dp.design

## Wirebonds

In [ ]:
design.delete_all_components()

In [ ]:
top_leftx = -5
top_lefty = 5
left_wb_xs = [1.6, 0.5-0.3]
left_wb_ys = [5-0.5+0.3]
left_wb_ys.append(5-1.6)
for i in range(8):
    new_y = left_wb_ys[-1]-0.85
    new_x = 0.5-0.3
    left_wb_xs.append(new_x)
    left_wb_ys.append(new_y)
# left_wb_ys.append(-5)

left_wb_xs = np.array(left_wb_xs)+top_leftx
left_wb_ys = np.array(left_wb_ys)

orientations_left = [270,0,0,0,0,0,0,0,0,0]

In [ ]:
top_rightx = 5
top_righty = 5
right_wb_xs = [5-1.6,4.5+0.3]
right_wb_ys = [5-0.5+0.3]
right_wb_ys.append(5-1.6)
for i in range(8):
    new_y = right_wb_ys[-1]-0.85
    right_wb_ys.append(new_y)
    new_x = 5-0.5+0.3
    right_wb_xs.append(new_x)
right_wb_ys = np.array(right_wb_ys)

orientations_right = [270,180,180,180,180,180,180,180,180,180]

In [ ]:
Wirebond_left_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='8um',
                           pad_width='200um',
                           pad_height='100um',
                           pad_gap='50um',
                           taper_height='50um')
                           
# Wirebond_left_opt = Dict(trace_width=design.variables['trace_width'],
#                            trace_gap=design.variables['trace_gap'],
#                            lead_length='25um',
#                            pad_width='150um',
#                            pad_height='200um',
#                            pad_gap='130um',
#                            taper_height='150um')

Place the wirebonds

In [ ]:
left_lauch_pads = []
for i in range(10):
    left_lauch_pads.append(LaunchpadWirebond(design,'wb_left'+str(i), options = Dict(pos_x = '{}mm'.format(left_wb_xs[i]),pos_y = '{}mm'.format(left_wb_ys[i]),orientation = orientations_left[i],**Wirebond_left_opt)))
    gui.rebuild()
    gui.autoscale()
    time.sleep(0.1)

In [ ]:
right_lauch_pads = []
for i in range(10):
    right_lauch_pads.append(LaunchpadWirebond(design,'wb_right'+str(i), options = Dict(pos_x = '{}mm'.format(right_wb_xs[i]),pos_y = '{}mm'.format(right_wb_ys[i]),orientation = orientations_right[i],**Wirebond_left_opt)))
    gui.rebuild()
    gui.autoscale()
    time.sleep(0.1)

In [ ]:
gui.rebuild()

## Qubit pockets

In [ ]:
from pocket import TransmonPocket as pocket

In [ ]:
pocket_options = dict(
        pos_x = '0mm', 
        pos_y = '0mm', 
        orientation = '0',
        frequency = 5.2,
        guess_path = r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv',
        coupling_path = '',
        sim = True,
        coord = '(0,0)',
        qubit_layer = 5,
        junction_layer = 2, 
        ab_layer = 8,
        ab_square_layer = 9,
        ab_distance = '70um',
        rotation = 90
        )

In [ ]:
qubit_y1 = ['3.96mm','3.86mm','2.910000mm','1.960000mm','1.010000mm',
            '0.060000mm','-0.890000mm','-1.840000mm','-2.790000mm','-3.740000mm']

First row numbers

In [ ]:
qubit_x0 = ['-3.9mm', '-3.1mm', '-2.2mm', 
            '-1.3mm', '-0.4mm', '0.5mm',
            '1.4mm', '2.3mm','3.2mm', '4.05mm']
qubit_y0 = ['4.5mm'] + ['3.96mm']*8 + ['4.2mm']
qubit_angles0 = [90,180,180,180,180,180,180,180,180,270]

### First row

qubits and qubit-qubit connections

In [ ]:
pockets = []#p1, p2]
for i in range(10):
    j = i
    pocket_options['pos_x'] = qubit_x0[j]
    pocket_options['pos_y'] = qubit_y0[j]
    pocket_options['coord'] = '({},0)'.format(j)
    pocket_options['rotation'] = qubit_angles0[j]
    p = pocket(gui, design,options = pocket_options)
    pockets.append(p)
    if i ==0:
        pass
    else:
        pockets[i].connect(pockets[i-1])
        # p.connect(pockets[-1])


Connecting the ends

In [ ]:
pockets[-1].connect(right_lauch_pads[0])
pockets[0].connect(left_lauch_pads[0])

### Rows 2-10

In [ ]:
xs = ['-4.3mm', '-3.35mm', '-2.4mm', '-1.45mm', '-0.5mm', '0.45mm', '1.4mm', '2.35mm', '3.3mm', '4.25mm']

In [32]:
# pockets = []#p1, p2]
for i in range(2,10):
    poc = []
    for j in range(10):
        pocket_options['pos_x'] = xs[j]
        pocket_options['pos_y'] = qubit_y1[i]
        pocket_options['coord'] = '({},{})'.format(j,i)
        pocket_options['rotation'] = 0 #qubit_angles0[j]
        p = pocket(gui, design,options = pocket_options)
        poc.append(p)
        if j ==0:
           poc[j].connect(left_lauch_pads[i]) 
        elif j == 9:
            poc[j].connect(poc[j-1])
            poc[j].connect(right_lauch_pads[i])
        else:
            poc[j].connect(poc[j-1])
    pockets.append(poc)

{'pos_x': '1.4mm', 'pos_y': '0.060000mm', 'orientation': '0', 'frequency': 5.2, 'guess_path': '/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv', 'coupling_path': '', 'sim': True, 'coord': '(6,5)', 'qubit_layer': 5, 'junction_layer': 2, 'ab_layer': 8, 'ab_square_layer': 9, 'ab_distance': '70um', 'rotation': 0}
1.4 -0.4
1.4 -0.5375000000000001


09:04AM 53s INFO [delete_component]: Called delete_component cpw_(6,5), but such a component is not in the design cache dictionary of components.


{'points': [array([ 1.8087 , -0.01138]), array([ 1.7963 , -0.01138])], 'middle': array([ 1.8025 , -0.01138]), 'normal': array([ 0., -1.]), 'tangent': array([ 1., -0.]), 'width': 0.0124, 'gap': 0.00744, 'chip': 'main', 'parent_name': 465, 'net_id': 0, 'length': 0} pin
{'start_pin': {'component': 'TQ(6,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(5,5)', 'pin': 'prime_end'}}
o {'anchors': {0: (1.2675, 0.06)}, 'trace_width': '12.4 um', 'trace_gap': '6.99 um', 'lead': {'start_straight': '5um', 'end_straight': '30um'}, 'fillet': '20um', 'total_length': '0.5mm', 'hfss_wire_bonds': True, 'q3d_wirebonds': True, 'pin_inputs': {'start_pin': {'component': 'TQ(6,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(5,5)', 'pin': 'prime_end'}}}
{'pos_x': '2.35mm', 'pos_y': '0.060000mm', 'orientation': '0', 'frequency': 5.2, 'guess_path': '/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv', 'coupling_path': '', 'sim': True, 'coord': '(7,5)', 'qubit_layer': 5, 'junctio

09:26AM 31s INFO [delete_component]: Called delete_component cpw_(7,5), but such a component is not in the design cache dictionary of components.


{'points': [array([ 2.7587 , -0.01138]), array([ 2.7463 , -0.01138])], 'middle': array([ 2.7525 , -0.01138]), 'normal': array([ 0., -1.]), 'tangent': array([ 1., -0.]), 'width': 0.0124, 'gap': 0.00744, 'chip': 'main', 'parent_name': 472, 'net_id': 0, 'length': 0} pin
{'start_pin': {'component': 'TQ(7,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(6,5)', 'pin': 'prime_end'}}
o {'anchors': {0: (2.2175, 0.06)}, 'trace_width': '12.4 um', 'trace_gap': '6.99 um', 'lead': {'start_straight': '5um', 'end_straight': '30um'}, 'fillet': '20um', 'total_length': '0.5mm', 'hfss_wire_bonds': True, 'q3d_wirebonds': True, 'pin_inputs': {'start_pin': {'component': 'TQ(7,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(6,5)', 'pin': 'prime_end'}}}
{'pos_x': '3.3mm', 'pos_y': '0.060000mm', 'orientation': '0', 'frequency': 5.2, 'guess_path': '/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv', 'coupling_path': '', 'sim': True, 'coord': '(8,5)', 'qubit_layer': 5, 'junction

...

10:11AM 51s INFO [delete_component]: Called delete_component cpw_(9,5), but such a component is not in the design cache dictionary of components.


{'points': [array([ 4.6587 , -0.01138]), array([ 4.6463 , -0.01138])], 'middle': array([ 4.6525 , -0.01138]), 'normal': array([ 0., -1.]), 'tangent': array([ 1., -0.]), 'width': 0.0124, 'gap': 0.00744, 'chip': 'main', 'parent_name': 486, 'net_id': 0, 'length': 0} pin
{'start_pin': {'component': 'TQ(9,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(8,5)', 'pin': 'prime_end'}}
o {'anchors': {0: (4.1175, 0.06)}, 'trace_width': '12.4 um', 'trace_gap': '6.99 um', 'lead': {'start_straight': '5um', 'end_straight': '30um'}, 'fillet': '20um', 'total_length': '0.5mm', 'hfss_wire_bonds': True, 'q3d_wirebonds': True, 'pin_inputs': {'start_pin': {'component': 'TQ(9,5)', 'pin': 'prime_start'}, 'end_pin': {'component': 'TQ(8,5)', 'pin': 'prime_end'}}}
1
{'start_pin': {'component': 'TQ(9,5)', 'pin': 'prime_end'}, 'end_pin': {'component': 'wb_right5', 'pin': 'tie'}}
o {'anchors': {0: (4.75, 0.06)}, 'trace_width': '12.4 um', 'trace_gap': '6.99 um', 'lead': {'start_straight': '5um', 'end_straigh

10:42AM 33s INFO [delete_component]: Called delete_component cpw_(0,6), but such a component is not in the design cache dictionary of components.


{'points': [array([-3.8913 , -0.96138]), array([-3.9037 , -0.96138])], 'middle': array([-3.8975 , -0.96138]), 'normal': array([ 0., -1.]), 'tangent': array([ 1., -0.]), 'width': 0.0124, 'gap': 0.00744, 'chip': 'main', 'parent_name': 495, 'net_id': 0, 'length': 0} pin


KeyboardInterrupt: 

In [36]:
for i in range(6,10):
    poc = []
    for j in range(10):
        pocket_options['pos_x'] = xs[j]
        pocket_options['pos_y'] = qubit_y1[i]
        pocket_options['coord'] = '({},{})'.format(j,i)
        pocket_options['rotation'] = 0 #qubit_angles0[j]
        p = pocket(gui, design,options = pocket_options)
        poc.append(p)
        if j ==0:
           poc[j].connect(left_lauch_pads[i]) 
        elif j == 9:
            poc[j].connect(poc[j-1])
            poc[j].connect(right_lauch_pads[i])
        else:
            poc[j].connect(poc[j-1])
    pockets.append(poc)

{'pos_x': '-4.3mm', 'pos_y': '-0.890000mm', 'orientation': '0', 'frequency': 5.2, 'guess_path': '/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv', 'coupling_path': '', 'sim': True, 'coord': '(0,6)', 'qubit_layer': 5, 'junction_layer': 2, 'ab_layer': 8, 'ab_square_layer': 9, 'ab_distance': '70um', 'rotation': 0}


In [35]:
gui.rebuild()

TypeError: MetalGUI.rebuild() got an unexpected keyword argument 'component'

In [ ]:
poc[j].connect(poc[j-1])

In [ ]:
poc

In [ ]:
poc[j].connect(right_lauch_pads[i])

In [ ]:
p_1.connect(right_lauch_pads[0])

In [ ]:
j = 8
pocket_options['pos_x'] = qubit_x0[j]
pocket_options['pos_y'] = qubit_y0[j]
pocket_options['coord'] = '({},0)'.format(j)
pocket_options['rotation'] = qubit_angles0[j]
p_2 = pocket(gui, design,options = pocket_options)

In [ ]:
qubit_angles0[9]

In [ ]:
p_2.connect(p_1)

In [ ]:
p.connect(pockets[8])

In [ ]:
p.connect(right_lauch_pads[0])

In [ ]:
# y_cord = ['3.96mm', '3.86mm']
y_cord = [3.96]

for i in range(9):
    # y_cord.append('{:2f}mm'.format(3.86-0.95*i))
    y_cord.append(np.round(3.86-0.95*i,2))
    print(3.86-0.95*i)

In [ ]:
y_cord

In [ ]:
wb1 = right_lauch_pads[0]
wb1.options.orientation = 0
gui.rebuild()

In [ ]:
pockets[-1].connect(p2)

In [ ]:
anchor = OrderedDict()
anchor[0] = (-3.2,4.00005)

In [ ]:
op  = {'anchors': anchor, 'trace_width': '12.4 um', 'trace_gap': '6.99 um', 'lead': {'start_straight': '5um', 'end_straight': '30um'}, 'fillet': '20um', 'total_length': '0.5mm', 'hfss_wire_bonds': True, 'q3d_wirebonds': True, 'pin_inputs': {'start_pin': {'component': 'TQ(2,0)', 'pin': 'prime_end'}, 'end_pin': {'component': 'TQ(1,0)', 'pin': 'prime_start'}}}

In [ ]:
dp.RouteMixed(design, 'aaa', 
           options = op)

In [ ]:
gui.rebuild()

In [ ]:
t1 = p1.Tee
t2= p2.Tee

In [ ]:
from collections import OrderedDict

In [ ]:
design.delete_component('aaa')

anchor = OrderedDict()
pin_inputs = Dict(
                start_pin=Dict(component=t1.name, pin='prime_end'),
                end_pin=Dict(component=t2.name, pin='prime_end'))
dp.trans_options['pin_inputs'] = pin_inputs
ops = Dict(anchors = anchor, **dp.trans_options)
# anchor[0] = (-3.87,3.5)
cpw = dp.RouteMixed(design, 'aaa', options = ops)


In [ ]:
design.delete_component('aaa')

In [ ]:
# design.delete_component('aaa')
gui.rebuild()

In [ ]:
design.delete_component('airbridge_connects(0,0)wb_left0')
design.delete_component('(0,0)CPWwb_left0')

In [ ]:
gui.rebuild()

In [ ]:
print(design.components)

In [ ]:
for  i in design.components:
    if 'pad' in type(i)

In [ ]:
wb_rights = []

In [ ]:
design.delete_all_components()

In [ ]:
pocket_options = dict(
        pos_x = '0mm', 
        pos_y = '0mm', 
        orientation = '0',
        frequency = 5.2,
        guess_path = r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv',
        coupling_path = '',
        sim = True,
        coord = '(0,0)',
        qubit_layer = 5,
        junction_layer = 2, 
        ab_layer = 8,
        ab_square_layer = 9,
        ab_distance = '70um',
        rotation = 90
        )

In [ ]:
design.delete_all_components()
pocket_options['pos_x'] = '0mm'
pocket_options['pos_y'] = '0mm'
pocket_options['coord'] = '(0,0)'

p1 = pocket(gui, design,options = pocket_options)

In [ ]:
pocket_options['pos_x'] = '1mm'
pocket_options['pos_y'] = '0mm'
pocket_options['coord'] = '(1,0)'

p2 = pocket(gui, design,options = pocket_options)

In [ ]:
p1.Tee.pins['second_end']

In [ ]:
p1.qubit.pins['a']

In [ ]:
gui.rebuild()
gui.autoscale()

In [ ]:
p1.connect(p2)

In [ ]:
qubit = p2.qubit
TQ = p2.Tee
j = p2.junction


In [ ]:
TQ

In [ ]:
f = p2.options['frequency']

In [ ]:
q1,j, cpw, TQ1, design,gui = dp.construct_cpw_qubit(qubit,j, TQ, f,gui,design,sim =False, displacement = '10mm')
q1, d = jj.place_junction(q1,j)
gui.rebuild()
gui.autoscale()

In [ ]:
a = Dict(q = 1, b = 2)

In [ ]:
a['q']

In [ ]:
'pocket' in str(type(p))

In [ ]:
p.options['coord']

In [ ]:
p.qubit.pins

In [ ]:
design = dp.design

junction_layer = 5
qubit_layer = 2
ab_layer = 7
ab_square_layer = 8

## Single Pad--Verify Design

### Draw the single pad transmon

In [ ]:
design.delete_all_components()

In [ ]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **dp.qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [ ]:
q1.options['junction'] = 'True'

In [ ]:
gui.rebuild()

In [ ]:
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'
gds = design.renderers.gds

In [ ]:
guesses = pd.read_csv(r'data/educated_guess_0403.csv')

In [ ]:
options = Dict(taper_len='0.5um',jj_gap = '0.14um')
d = junction(design, 'd', options = options)


In [ ]:
k = 3

q1.options['pad_height'] = '{}um'.format(guesses['Sizes (um)'].values[k])
q1.options['pad_width'] = '{}um'.format(guesses['Sizes (um)'].values[k])

q1.options['pocket_width'] = '{}um'.format(guesses['Sizes (um)'].values[k]+2*gap.to(u.um).value)

q1.options['connection_pads']['a']['pad_width'] = guesses['Coupling_len'][k]
q1.options['connection_pads']['a']['pad_height'] = '30um-{}um'.format(guesses['Coupling_gap(um)'][k])
q1.options['connection_pads']['a']['pad_gap'] = '{}um'.format(guesses['Coupling_gap(um)'][k])

d.options['Lj'] = '{}'.format(guesses['Ljs (nH)'][k])

q1.options.layer = qubit_layer
d.options.layer = junction_layer

gui.rebuild()
gui.zoom_on_components(['Q1'])
    

In [ ]:
q1, d = jj.place_junction(q1,d)
gui.rebuild()


In [ ]:
dp.TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.1 mm',
                                             pos_y='0.5 mm',
                                             mirror = True, **dp.TQ_options))
TQ1.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

In [ ]:
180//90

In [ ]:
q1,j, cpw, TQ1, design,gui = dp.construct_cpw_qubit(q1,d, TQ1, 5,gui,design,sim =False)
q1, d = jj.place_junction(q1,d)
gui.rebuild()
gui.autoscale()

Construct the cpw airbridge object

In [ ]:
design.delete_component('airbridges')

In [ ]:
ab_options = Dict(cpw_name = 'cpw_1', distance = '100um', dis = '10um', layer_ab_square = str(ab_square_layer), layer_ab = str(ab_layer), total_length = '80 um', chip = 'main', seg_num = '0')

In [ ]:
ab(design, 'airbridges', ab_options)

In [ ]:
gui.rebuild()

In [ ]:
q1.options.layer = qubit_layer
cpw.options.layer = qubit_layer
TQ1.options.layer = qubit_layer
j.options.layer = junction_layer
gui.rebuild()
gui.autoscale()

In [ ]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '10mm'
gui.rebuild()
a_gds = design.renderers.gds

In [ ]:
a_gds.options.cheese.edge_nocheese = '300um'
a_gds.options.no_cheese.buffer = '50um'

In [ ]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True

In [ ]:
a_gds.export_to_gds('export_test.gds')

In [ ]:
a_gds = design.renderers.gds

In [ ]:
a_gds.options.tolerance = '0.00001'
a_gds.options.cheese.edge_nocheese = '300um'
a_gds.options.no_cheese.buffer = '40um'

In [ ]:
a_gds.options

In [ ]:
a_gds.options['path_filename'] = 'Customized_Component/Fake_Junctions.GDS'

In [ ]:
a_gds.options.no_cheese.buffer = '0um'

a_gds.options['cheese']['view_in_file']['main'][5] = False
a_gds.options['no_cheese']['view_in_file']['main'][5] = False
a_gds.options['ground_plane'] = True

In [ ]:
q1.options.layer = 1
d.options.layer = 5
gui.rebuild()

In [ ]:
a_gds.options['ground_plane'] = True

In [ ]:

# a_gds.options['short_segments_to_not_fillet'] = True
# a_gds.options['check_short_segments_by_scaling_fillet'] = 1.0
a_gds.export_to_gds('Dolan.gds')#, highlight_qcomponents=['TQ1'])

In [ ]:
a_gds.options

In [ ]:
design.components.keys()

In [ ]:
d = {}


In [ ]:
base = 822.76
dose = 0.8
for j in range(5):
    d['Column' + str(j+1)] = []
    dose = 0.8 + j*0.1
    for i in range(10):
        dose1 = dose + 0.01*i
        d['Column' + str(j+1)].append(base*round(dose1,2))
    
    


In [ ]:
matrix = pd.DataFrame(d)

In [ ]:
matrix.index += 1

In [ ]:
matrix